In [4]:
import numpy as np

dist_mat = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        dist_mat[i,j] = min(i+1,j+1,5-i, 5-j)
dist_mat = np.exp(dist_mat)/np.sum(np.exp(dist_mat))
print(dist_mat)

[[0.0221556  0.0221556  0.0221556  0.0221556  0.0221556 ]
 [0.0221556  0.06022517 0.06022517 0.06022517 0.0221556 ]
 [0.0221556  0.06022517 0.16370899 0.06022517 0.0221556 ]
 [0.0221556  0.06022517 0.06022517 0.06022517 0.0221556 ]
 [0.0221556  0.0221556  0.0221556  0.0221556  0.0221556 ]]
1.0


In [12]:
test = [[1,1,1,1,1], [1,1,1,1,1], [2,2,2,2,2], [1,1,1,1,1], [0,0,0,0,0]]
print(test)
print(test * dist_mat)
print(np.sum(test), np.sum(test * dist_mat))

[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [2, 2, 2, 2, 2], [1, 1, 1, 1, 1], [0, 0, 0, 0, 0]]
[[0.0221556  0.0221556  0.0221556  0.0221556  0.0221556 ]
 [0.0221556  0.06022517 0.06022517 0.06022517 0.0221556 ]
 [0.0443112  0.12045034 0.32741798 0.12045034 0.0443112 ]
 [0.0221556  0.06022517 0.06022517 0.06022517 0.0221556 ]
 [0.         0.         0.         0.         0.        ]]
25 1.21769252493608


In [ ]:
# from model.recon import *